# Query Organization Orders
This script allows a user with the organization manager role to pull the orders in an organization and view the products associated with each order in a table.

The cells that require the user to input parameters prior to running are marked with `USER INPUT REQUIRED` in the header. Searching for `USER` in the notebook will also identify where the user must enter inputs.

* **Author:** Hayley Pippin
* **Last updated:** August 9, 2024
* **Required input(s):**
    * Organization ID for the organization of interest
    * `credentials.json`: JSON containing the user's Capella Console credentials. *Note: The user must be an organization manager of the organization they are pulling orders from*.
* **Output(s):**
    * NA

## Setup


### Install packages
The following cell **only needs to be run once** if packages are not already installed. Uncomment any of the following lines to install the necessary packages.

In [ ]:
#!pip install requests
#!pip install json
#!pip install pandas

### Import packages and define helper functions


In [ ]:
import requests
import json
import pandas as pd

# Function to view printed JSON files easier
def p(data):
        print(json.dumps(data, indent=2))

### Authentication (INITIAL USER INPUT REQUIRED)
This cell needs to be run hourly to re-authenticate with the Capella system.

In [ ]:
# Load username and password
with open('WRITE PATH TO CREDENTIALS FILE HERE') as f: # USER: Input path to credentials.json file. User MUST be an organization manager of the organization of interest.
    data = json.load(f)
    username = data['username']
    password = data['password']

# Get a valid token from the auth service
r = requests.post("https://api.capellaspace.com/token", 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))
access_token = r.json()["accessToken"]
# p(accessToken)

# GET user ID and org ID
headers = {'Authorization':'Bearer ' + access_token}
r = requests.get("https://api.capellaspace.com/user", headers=headers)
user_id = r.json()["id"]
org_id = r.json()["organizationId"]
#p(r.json())

# Print user and org ID
print('User email: ', r.json()['email'], '\nOrganization: ', r.json()['organization']['name'], '\nEnvironment: ', r.json()['apiEnvironmentRole'])

## Inputs (USER INPUT REQUIRED)
Define the organization ID to pull the order from.

In [ ]:
r = requests.get("https://api.capellaspace.com/organization", headers=headers) # Get information about the user's organization. User MUST be an organization manager to use this endpoint.
# p(r.json()) # Uncomment to see all details of organization.

In [ ]:
# USER: Define organization ID to pull orders from.
# org_id = r.json()["id"] # OPTION 1: Use the ID associated with the organization that the user is currently in.
# org_id = '' # OPTION  2: Manually define the org ID the user wants to pull the orders from.

params = {
    'organizationId': org_id,
    # 'customerId': user_id
    # 'resellerId': org_id, # Use this option if organization is a reseller to pull all orders from sub-organizations. WARNING: Currently experiencing timeout issues.
    'limit': 100 # Set limit for number of returns to avoid timeout.
}

## Get List of Orders and Associated Collects

In [ ]:
r = requests.get("https://api.capellaspace.com/orders/", headers=headers, params=params)

In [ ]:
# Format and view dataframe of orderd
df = pd.json_normalize(r.json())
df = df.explode('items').reset_index(drop = True)
df = df.rename(columns = {'items': 'order_items'})
df = pd.concat([df, df.order_items.apply(pd.Series)], axis = 1).groupby(['orderId', 'collectId'], group_keys=True).apply(lambda x: x).drop(['userId', 'organizationId', 'orderId', 'contractId', 'expirationDate', 'order_items', 'itemStatus', 'collectionId', 'collectIds', 'collectId', 'size'], axis = 1)
df

In [ ]:
df.to_csv('WRITE EXPORT PATH HERE') # USER: Input path and file name for export here.